# BERT의 MLM을 사용한 Classification

BERT Pretrain 과정에서는 masked token을 맞추는 Masked Language Modeling이라는 방식을 사용합니다.  
예를 들어, "my dog is hairy" 라는 문장이 있을 때, "my dog is `[MASK]`" 로 문장을 변경하고, 모델이 `[MASK]` 토큰을 `hairy`라고 예측할 수 있게 학습하는 것입니다.  

이 방식을 Classification에도 사용할 수 있습니다.   
"인천→핀란드 항공기 결항…휴가철 여행객 분통" 문장의 label은 `세계` 입니다.   
"인천→핀란드 항공기 결항…휴가철 여행객 분통" 문장의 뒤에 "[SEP]이 문장은 `[MASK]`"라는 문장을 붙였습니다.  
그 후 모델이 `[MASK]`를 `세계`로 예측하도록 학습시켰습니다.  


In [1]:
import random
from tqdm.notebook import tqdm, tnrange
import os

import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForPreTraining

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler

from sklearn.metrics import accuracy_score

if torch.cuda.is_available():
    print("사용가능한 GPU수 : ",torch.cuda.device_count())
    device = torch.device("cuda")
else:
    print("CPU 사용")
    device = torch.device("cpu")

사용가능한 GPU수 :  1


Reproduction을 위한 Seed 고정  
출처 : https://dacon.io/codeshare/2363?dtype=vote&s_id=0

In [2]:
RANDOM_SEED = 42

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(RANDOM_SEED)

모델은 klue/bert-base을 사용하였습니다.

In [3]:
model_checkpoint = "klue/bert-base"
save_checkpoint_path = "./checkpoints"
batch_size = 32

huggingface 에서 tokenizer를 불러옵니다.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

huggingface 에서 model를 불러옵니다.  
`AutoModelForSequenceClassification`이 아닌 `AutoModelForPreTraining`을 사용합니다.  

In [5]:
model = AutoModelForPreTraining.from_pretrained(model_checkpoint)

In [6]:
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

dataset을 가져옵니다. 

In [7]:
dataset = pd.read_csv("data/train_data.csv")
test = pd.read_csv("data/test_data.csv")

In [8]:
dataset.head(3)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4


데이터를 MLM Classification 방식에 맞게 전처리해야합니다.  
`[SEP]이 문장은 [MASK]` 문장을 모든 데이터 뒤에 붙여줍니다. 

In [9]:
for idx in range(len(dataset["title"])):
    dataset["title"].iloc[idx] += ".[SEP] 이 문장은 [MASK]"

for idx in range(len(test["title"])):
    test["title"].iloc[idx] += ".[SEP] 이 문장은 [MASK]"

D:\Workspace\guan\sentimental_classification\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
dataset.head(3)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통.[SEP] 이 문장은 [MASK],4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화.[SEP] 이 문장은 [MASK],4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것.[SEP] 이 문장은 [MASK],4


klue/bert-base의 Tokenizer에서 각 라벨들의 토큰을 확인합니다.  
하나의 토큰으로 예측을 하기 위해 `IT과학` -> `과학`으로, `생활문화` -> `문화`로 바꿨습니다.   

`과학`은 4038  
`경제`는 3674  
`사회`는 3647  
`문화`는 3697  
`세계`는 3665  
`스포츠`는 4559  
`정치`는 3713

입니다. 

In [11]:
topic_token_dict = {0:4038,1:3674,2:3647,3:3697,4:3665,5:4559,6:3713}
token_topic_dict = {4038 : 0, 3674 : 1, 3647 : 2, 3697 : 3, 3665 : 4, 4559 : 5, 3713 : 6}
topic_dict = {0: "과학", 1:"경제", 2:"사회", 3:"문화", 4:"세계", 5:"스포츠", 6 : "정치"}

train과 validation을 나눠줍니다. 

In [12]:
dataset_train, dataset_val = train_test_split(dataset,test_size = 0.2,random_state = RANDOM_SEED)

In [13]:
dataset_train.head()

,index,title,topic_idx
4824,4824,KT 화상회의 가능한 기업용 전화 출시.[SEP] 이 문장은 [MASK],1
21714,21714,문 대통령 해리 해리스 주한 미국대사와 함께.[SEP] 이 문장은 [MASK],6
29145,29145,김기식 사퇴 찬성 51%…文대통령 지지율 66.2%로 하락리얼미터.[SEP] 이 문...,6
18342,18342,민주콩고서 수백명 에볼라 사태 끝내자 거리행진.[SEP] 이 문장은 [MASK],4
9010,9010,충남 올겨울 첫 한파주의보…눈 최대 15㎝ 쌓일 듯.[SEP] 이 문장은 [MASK],3


입력 문장을 tokenize 해줍니다. 
이때, [MASK]토큰의 위치를 기억해줍니다.

In [14]:
def bert_tokenize(dataset,sent_key,label_key,tokenizer):
    if label_key is None :
        labels = [np.int64(0) for i in dataset[sent_key]]
    else :
        labels = [np.int64(i) for i in dataset[label_key]]
    
    sentences = tokenizer(dataset[sent_key].tolist(),truncation=True,padding=True)

    input_ids = sentences.input_ids
    token_type_ids = sentences.token_type_ids
    attention_mask = sentences.attention_mask
    
    # [MASK] 토큰의 인덱스 저장. 
    masked_token_idx = []
    for input_id in input_ids:
        masked_token_idx.append(input_id.index(4))
    
    return list([input_ids, token_type_ids, attention_mask, labels, masked_token_idx])

In [15]:
train_inputs = bert_tokenize(dataset_train,"title","topic_idx",tokenizer)
validation_inputs = bert_tokenize(dataset_val,"title","topic_idx",tokenizer)
test_inputs = bert_tokenize(test,"title",None,tokenizer)

토큰화가 잘 되었습니다. 

In [16]:
print(dataset_train["title"][0])
print(train_inputs[0][0])

인천→핀란드 항공기 결항…휴가철 여행객 분통.[SEP] 이 문장은 [MASK]
[2, 5108, 10948, 7288, 3662, 2470, 3646, 2048, 4117, 4542, 18, 3, 1504, 6265, 2073, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


padding된 데이터의 길이를 확인해줍니다. 

In [17]:
print(len(test_inputs[0][1]), len(validation_inputs[0][1]), len(train_inputs[0][1]))
max_length = max(len(validation_inputs[0][1]), len(train_inputs[0][1]))
print("max length :",max_length)

36 35 35
max length : 35


In [18]:
for i in range(len(train_inputs)):
    train_inputs[i] = torch.tensor(train_inputs[i])
    
for i in range(len(validation_inputs)):
    validation_inputs[i] = torch.tensor(validation_inputs[i])
    
for i in range(len(test_inputs)):
    test_inputs[i] = torch.tensor(test_inputs[i])

In [19]:
train_data = TensorDataset(*train_inputs)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(*validation_inputs)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

test_data = TensorDataset(*test_inputs)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

hyper parameter를 설정해줍니다. 

In [20]:
lr = 2e-5
adam_epsilon = 1e-8
epochs = 5

num_warmup_steps = 0

warmup_ratio = 0.1
num_training_steps = len(train_dataloader)*epochs
warmup_step = int(num_training_steps * warmup_ratio)

optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon) 
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=num_training_steps) 

학습을 진행합니다.  
`AutoModelForPreTraining`에 데이터를 입력했을 때 나오는 output은 이 문장이 다음 문장인지 아닌지를 판단하는 확률과 [MASK] 토큰을 예측한 값입니다.  
output과 정답에 대한 Loss를 계산하고 Loss를 줄이는 방향으로 학습이 진행됩니다.  

In [21]:
train_loss_set = []
# learning_rate = []

criterion_lm = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean')
criterion_cls = torch.nn.CrossEntropyLoss()

model.zero_grad()

for _ in tnrange(1,epochs+1,desc='Epoch'):
    print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
    batch_loss = 0
    
    # train
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_masked_token_idx = batch
        
        outputs = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)
        
        # calculate loss
        logits_cls, logits_lm = outputs[1], outputs[0]
        
        # topic -> token 
        # Ex.6 -> 3713
        mask_label = [topic_token_dict[lb] for lb in b_labels.to('cpu').numpy() ]
        
        labels_lms = []
        
        # label 만들기
        for idx, label in zip(b_masked_token_idx.to('cpu').numpy(),mask_label):
            labels_lm = np.full(max_length, dtype=np.int, fill_value=-1)
            labels_lm[idx] = label
            labels_lms.append(labels_lm)
        label_lms_pt = torch.tensor(labels_lms,dtype=torch.int64).to(device)
        
        # lm loss 계산
        loss_lm = criterion_lm(logits_lm.view(-1, logits_lm.size(2)), label_lms_pt.view(-1))
        
        # cls loss 계산
        labels_cls = [1 for _ in range(len(b_input_ids))]
        labels_cls = torch.tensor(labels_cls).to(device)
        loss_cls = criterion_cls(logits_cls, labels_cls)
        
        loss = loss_cls + loss_lm
        loss.backward()        
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        scheduler.step()
        optimizer.zero_grad()

        batch_loss += loss.item()
        
    avg_train_loss = batch_loss / len(train_dataloader)
    train_loss_set.append(avg_train_loss)
    print(F'\n\tAverage Training loss: {avg_train_loss}')
    
    # eval
    model.eval()
    
    predict_list = []
    label_list = []
    
    for step, batch in enumerate(tqdm(validation_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_masked_token_idx = batch

        outputs = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)

        logits_cls, logits_lm = outputs[1], outputs[0]
        
        # 예측한 값.
        logits_lm_np = logits_lm.to('cpu').detach().numpy()
        pred = np.argmax(logits_lm_np,axis=2)
        
        # [MASK] 토큰의 인덱스
        masked_token_idx_np = b_masked_token_idx.to('cpu').numpy()
        
        # label
        labels_np = b_labels.to('cpu').numpy()

        for i in range(len(pred)):
            # 예측한 토큰을 label로 바꾸기
            l = token_topic_dict[pred[i][masked_token_idx_np[i]]]
            predict_list.append(l)

        for l in labels_np:
            # 정답을 넣기
            label_list.append(l)

        # topic -> token 
        # Ex.6 -> 3713
        mask_label = [ topic_token_dict[lb] for lb in labels_np ]
        
        labels_lms = []
        for idx, label in zip(masked_token_idx_np,mask_label):
            
            labels_lm = np.full(max_length, dtype=np.int, fill_value=-1)
            labels_lm[idx] = label
            labels_lms.append(labels_lm)

        #mlm loss 계산
        labels_lms_pt = torch.tensor(labels_lms,dtype=torch.int64).to(device)
        loss_lm = criterion_lm(logits_lm.view(-1, logits_lm.size(2)), labels_lms_pt.view(-1))

        #nsp loss 계산
        labels_cls = [1 for _ in range(len(b_input_ids))]
        labels_cls = torch.tensor(labels_cls).to(device)
        loss_cls = criterion_cls(logits_cls, labels_cls)

        loss = loss_cls + loss_lm
        
        batch_loss += loss.item()

    print("\n\tAccuracy : {}".format(accuracy_score(predict_list,label_list)))
    avg_validation_loss = batch_loss / len(validation_dataloader)
    print(F'\n\tAverage validation loss: {avg_validation_loss}')

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>


  0%|          | 0/1142 [00:00<?, ?it/s]


	Average Training loss: 0.7418069830833038


  0%|          | 0/286 [00:00<?, ?it/s]

...


	Accuracy : 0.8923447596101194

	Average validation loss: 0.9058767256206488
<====================== Epoch 5 ======================>


  0%|          | 0/1142 [00:00<?, ?it/s]


	Average Training loss: 0.07596644382413531


  0%|          | 0/286 [00:00<?, ?it/s]


	Accuracy : 0.893549446938999

	Average validation loss: 0.8153319107517161


훈련이 끝나고, 예측을 진행합니다.

In [25]:
predict_li = []

model.eval()
for step, batch in enumerate(tqdm(test_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_masked_token_idx = batch
    
    outputs = model(b_input_ids, token_type_ids=b_token_type_ids, attention_mask=b_input_mask)
    
    logits_cls, logits_lm = outputs[1], outputs[0]
    
    logits_lm = logits_lm.to('cpu').detach().numpy()
    out = np.argmax(logits_lm,axis=2)
    
    masked_token_idx_np = b_masked_token_idx.to('cpu').numpy()
    
#     print(out)
    
    for i in range(len(out)):
        l = token_topic_dict[out[i][masked_token_idx_np[i]]]
        predict_li.append(l)

  0%|          | 0/286 [00:00<?, ?it/s]

In [23]:
len(predict_li)

9131

In [24]:
submission = pd.read_csv('data/sample_submission.csv')
submission['topic_idx'] = predict_li
submission.to_csv("results/klue-bert-mlm-classification.csv",index=False)